In [1]:
import sys
sys.path.append('..')
import tedq as qai

import pennylane as qml

# Related package
import torch
import numpy as np
import matplotlib.pyplot as plt
import time

import gpytorch


# Hamiltonian related
from openfermion.chem import MolecularData
import openfermion
from openfermionpyscf import run_pyscf

# MISC
r_bohr = 0.529177210903

# Global variable
n_qubits = 4
n_layers = 3
n_search = 500
n_experts = 5

# 
tolerance = 1e-6
min_pass = 5

# Set molecule parameters.
basis = 'sto-3g'
multiplicity = 1
n_points = 40
bond_length_interval = 3.0 / n_points

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
import itertools


valid_Rs =  [qai.RY, qai.RZ]
valid_CNOTs = ([0, 1], [1, 2], [2, 3])

Rs_space = list(itertools.product(valid_Rs, valid_Rs, valid_Rs, valid_Rs))
CNOTs_space = [[y for y in CNOTs if y is not None] for CNOTs in list(itertools.product(*([x, None] for x in valid_CNOTs)))]
NAS_search_space = list(itertools.product(Rs_space, CNOTs_space))

In [3]:
len(NAS_search_space)

128

In [4]:
NAS_search_space[3][1]

[[0, 1]]

In [5]:
def Rot(alpha, beta, theta, qubit):
    qml.RX(alpha, wires=qubit)
    qml.RY(beta, wires=qubit)
    qml.RZ(theta, wires=qubit)
# Ansatz
def ansatz(params):
    for i in range(n_qubits):
        Rot(params[i][0], params[i][1], params[i][2], i)
    for j in range(n_qubits-1, -1, -1):
        for k in range(j+1, n_qubits):
            qml.CNOT(wires=[j, k])

In [6]:
# Hamiltonian


def get_H2_hamiltonian(distance):
    bond_length=distance*r_bohr
    geometry = [('H', (0., 0., -bond_length/2)), ('H', (0., 0., bond_length/2))]
    molecule = MolecularData(
        geometry, basis, multiplicity,charge=0,
        description=str(round(bond_length, 2)))
    molecule = run_pyscf(molecule,run_scf=1,run_fci=1)
    terms_molecular_hamiltonian = molecule.get_molecular_hamiltonian(occupied_indices=[], active_indices=[0,1])
    fermionic_hamiltonian = openfermion.transforms.get_fermion_operator(terms_molecular_hamiltonian)
    jw_hamiltonian = openfermion.transforms.jordan_wigner(fermionic_hamiltonian)
#     print(jw_hamiltonian.terms)
#     print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))
#     print('Hartree-Fock energy of {} Hartree.'.format(molecule.fci_energy))

    return jw_hamiltonian.terms, molecule.fci_energy


In [7]:
AIdList = []

# Ansatz

def ansatz(params, AIdList):
    for i, Aid in enumerate(AIdList):
        for j in range(n_qubits):
            NAS_search_space[Aid][0][j](params[i][j], qubits=[j])
        for cnot_pair in NAS_search_space[Aid][1]:
            qai.CNOT(qubits=list(cnot_pair))

In [8]:

def getGateFromName(name):
    if name=="I":
        return qai.I
    if name=="X":
        return qai.PauliX
    if name=="Y":
        return qai.PauliY
    if name=="Z":
        return qai.PauliZ
def measurements(gatesPrefix):
    gateList = []
    for qubit, gatePrefix in list(gatesPrefix):
        gate = getGateFromName(gatePrefix)
        gateList.append(gate(qubits=[qubit]))
    return gateList

def initCircuit(distance, selectedAIdList):
    AIdList = selectedAIdList
    H, fci_energy = get_H2_hamiltonian(distance)
    for idx in H:
        def circuitDef(params):
            ansatz(params, AIdList)
            qai.measurement.expval(measurements(idx))
        circList.append(qai.Circuit(circuitDef, n_qubits, torch.zeros(n_layers,n_qubits)))
        compiledCircList.append(circList[-1].compilecircuit('pytorch', print_output=False))
        weightList.append(H[idx])
    return fci_energy


In [9]:
def expert_evaluator(model, subnet, n_experts, cost_fn):

    target_expert = 0
    target_loss = None
    for i in range(n_experts):
        model.params = model.get_params(subnet, i)
        temp_loss = cost_fn(model.params)
        if target_loss is None or temp_loss < target_loss:
            target_loss = temp_loss
            target_expert = i
    return target_expert

In [10]:
class CircuitSearchModel():
    def __init__(self, n_qubits=4, n_layers=3, n_experts=5):
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.n_experts = n_experts
        '''init params'''
        # randomly initialize parameters from a normal distribution
        self.params_space = np.random.uniform(0, np.pi * 2, (n_experts, n_layers, len(Rs_space), n_qubits))
        self.params = torch.ones((3,4), requires_grad=True)
    
    def get_params(self, subnet, expert_idx):
        self.subnet = subnet
        self.expert_idx = expert_idx
        params = []
        # print("CNOT", NAS_search_space)
        # print("get", subnet, expert_idx)
        for j in range(self.n_layers):
            r_idx = subnet[j] // len(CNOTs_space)
            params.append(self.params_space[expert_idx, j, r_idx:r_idx+1])
#         print("get param", params, np.concatenate(params, axis=0))
        self.params=torch.tensor(params, requires_grad=True)
        return np.concatenate(params, axis=0)

    def set_params(self):
        self.params = self.params.cpu().detach().numpy()
        for j in range(self.n_layers):
            r_idx = self.subnet[j] // len(CNOTs_space)
            # print("set", j, self.subnet[j], r_idx, len(CNOTs_space))
            # print("set", self.params_space)
            self.params_space[self.expert_idx, j, r_idx:r_idx+1] = self.params[j, :]

    def __call__(self, params, wires):
        circuit_search(params, wires=wires,
                       n_qubits=self.n_qubits,
                       n_layers=self.n_layers,
                       arch=self.subnet)

In [11]:
import torch.nn as nn

distList = np.arange(0.5,1.0, 0.5)
energyList = np.array([])
timeList = np.array([])
fciEnergyList = np.array([])
modelParam = CircuitSearchModel()

timestamp = np.array([])
error = np.array([])

distance = 1.2



In [12]:
step_size = 0.2


time_start=time.time()    


for i_iter in range(n_search):
    circList = []
    compiledCircList = []
    weightList = []
    selectedAIdList = np.random.randint(0, len(NAS_search_space), (n_layers,)).tolist()
    
    fciE = initCircuit(distance, selectedAIdList)
    
#     find optimal expert
    expert_idx = np.random.randint(n_experts)
    if i_iter>100:
        for i in range(n_experts):
            # get params
            min_loss = 100000
            min_expert_id = 0
            params = modelParam.get_params(selectedAIdList, i)
#             print("params",params)
            # calculate
            exp = 0
            for idx, compiledCirc in enumerate(compiledCircList):   
#                 print("compile idx ", idx)
                exp += compiledCirc(modelParam.params).real*weightList[idx].real
            loss = exp
            if loss<min_loss:
                min_loss = loss
                min_expert_id = i
        expert_idx = min_expert_id
    # get params
    modelParam.get_params(selectedAIdList, expert_idx)
#     params = torch.zeros((3,4))
    vNgdOpt = gpytorch.optim.NGD([modelParam.params], num_data=1, lr=0.1)
    
    # for each set
    l_sum = 0
    loss = nn.L1Loss()
    exp = 0
    for idx, compiledCirc in enumerate(compiledCircList):   
        exp += compiledCirc(modelParam.params).real*weightList[idx].real


    l = loss(exp, torch.Tensor([-100.]))
    l.backward()        
#     optimizer.step()
#     optimizer.zero_grad()
#     params = opt.step(cost, params)
#     print("old", modelParam.params)
    vNgdOpt.step()
#     print("new", modelParam.params)
#     print("grad", modelParam.params.grad)
    vNgdOpt.zero_grad()

    # set params
    modelParam.set_params()
        
        
#     print(params)
    print(i_iter, selectedAIdList, fciE, exp.item())
    energyList = np.append(energyList, exp.item())
    timeList = np.append(timeList, time.time()-time_start)
    fciEnergyList = np.append(fciEnergyList, fciE)
time_end=time.time()

/var/folders/45/8c2bcn7n3ln498zczt81kn9c0000gp/T/ipykernel_70411/4090894739.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  self.params=torch.tensor(params, requires_grad=True)


0 [69, 118, 20] -1.1266988216513705 0.2514457404613495
1 [82, 101, 57] -1.1266988216513705 0.4698590934276581
2 [84, 13, 68] -1.1266988216513705 0.5829383730888367
3 [24, 99, 29] -1.1266988216513705 0.06856957077980042
4 [102, 65, 72] -1.1266988216513705 -0.22568930685520172
5 [41, 70, 68] -1.1266988216513705 0.48540639877319336
6 [62, 107, 8] -1.1266988216513705 0.5406479835510254
7 [45, 8, 117] -1.1266988216513705 -0.2472575157880783
8 [44, 36, 15] -1.1266988216513705 -0.3995998203754425
9 [115, 113, 71] -1.1266988216513705 0.21514521539211273
10 [32, 92, 21] -1.1266988216513705 0.3580686151981354
11 [83, 29, 26] -1.1266988216513705 0.3822665214538574
12 [126, 80, 38] -1.1266988216513705 0.10344864428043365
13 [66, 53, 50] -1.1266988216513705 0.6490551829338074
14 [85, 106, 127] -1.1266988216513705 -0.3903382122516632
15 [108, 47, 20] -1.1266988216513705 0.28957754373550415
16 [60, 50, 39] -1.1266988216513705 0.7226638197898865
17 [44, 65, 6] -1.1266988216513705 -0.2958071231842041
1

compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[3.82531919e+00 9.74054996e-01 8.38973460e-01 3.40012443e-03]
 [1.03845545e+00 4.66864915e+00 2.82627336e+00 1.77784552e+00]
 [5.84712655e+00 3.83975939e+00 4.72032901e-01 3.91733978e+00]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
102 [24, 79, 60] -1.1266988216513705 -0.21896561980247498
params [[0.89235825 1.893167   1.17211904 3.77248992]
 [5.85252338 6.12760989 3.30432126 5.79418275]
 [0.15873839 2.71823637 2.25890065 5.10106676]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[2.55772771 1.75451878 1.47549959 4.62748958]
 [0.37572025 5.800

params [[0.89235825 1.893167   1.17211904 3.77248992]
 [1.5309624  4.68151591 4.30385005 5.90567567]
 [6.16888444 3.28034821 3.16365742 1.69612754]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[2.55772771 1.75451878 1.47549959 4.62748958]
 [2.18131759 2.67023542 4.96170103 3.33464334]
 [0.84609589 0.59320011 0.32089823 2.10399036]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[4.41484896 5.81017969 4.96128876 3.96512901]
 [6.18916581 5.33483929 5.56555244 3.53468423]
 [3.0523582  3.63209088 4.16891389 5.28560278]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
com

111 [111, 8, 72] -1.1266988216513705 0.3188846707344055
params [[5.53253876 0.64563503 2.48035477 1.76872739]
 [0.70546889 1.75407116 2.51156641 2.03524646]
 [0.25026207 2.94688677 2.95941977 4.32380585]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[0.51691877 4.54154804 5.94395411 1.61060742]
 [4.63980694 1.79035835 3.31562889 0.2373397 ]
 [4.15299062 0.55274867 5.63218349 1.4206322 ]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[5.06309249 2.95239971 4.5173062  4.01746009]
 [4.36026928 2.48855582 3.71162322 4.1747455 ]
 [3.14341332 3.90662681 5.8108552  5.61153647]]
compile idx  0
compile idx  1
compile

...

params [[2.5109847  4.9142247  2.23074953 2.76197628]
 [5.3972584  4.23272336 5.72451553 3.74147027]
 [1.48995922 5.62956656 4.96914749 2.79294082]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[2.1973005  4.0921528  5.90633896 4.76829576]
 [3.85960056 6.17976677 0.37452829 3.13812537]
 [0.93780205 5.27608866 5.99577194 1.47328082]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[5.48047652 4.71504842 2.38779447 4.69031967]
 [0.53368208 2.26815573 0.85584838 3.36783596]
 [4.15557658 1.53675954 0.62502092 2.10135598]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
com

params [[3.15141421 0.81849292 3.37373702 0.34855704]
 [1.9723876  1.09331173 3.65947024 5.77420846]
 [5.81709814 1.20573762 2.83537604 1.185955  ]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[2.00522324 4.37238962 1.98754202 4.4137233 ]
 [1.70406268 3.30780347 2.1820573  2.10108599]
 [1.11143794 4.59288688 1.38247111 3.73572936]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
399 [65, 67, 109] -1.1266988216513705 0.27486780285835266
params [[5.53253876 0.64563503 2.48035477 1.76872739]
 [2.52680993 4.96992397 3.61739658 5.97272355]
 [4.92114009 5.1348117  2.52972092 1.65682695]]
compile idx  0
compile idx  1
compi

compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
403 [43, 86, 64] -1.1266988216513705 -0.1994134783744812
params [[1.39386228 1.45258834 4.06715637 2.66752681]
 [0.66452493 0.04733511 2.66576474 3.28841585]
 [1.27913256 1.98143733 0.14805772 2.01577153]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[0.87734782 3.19153959 0.75216797 2.30672167]
 [5.96571498 4.53394149 2.05942221 2.66301848]
 [1.73680491 4.97824416 3.81225584 3.96514792]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[0.4353

compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
408 [90, 109, 29] -1.1266988216513705 -0.5160077214241028
params [[1.66444565 2.68774882 0.38866153 3.44284514]
 [2.64512358 5.10468301 1.38801571 0.50433468]
 [0.99017472 3.07995464 1.64419564 1.8851255 ]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14
params [[1.5272134  4.43818284 1.00316574 5.84281865]
 [2.21975174 4.90565054 0.75795775 5.44891102]
 [3.83375937 5.71536015 3.84436226 2.45871601]]
compile idx  0
compile idx  1
compile idx  2
compile idx  3
compile idx  4
compile idx  5
compile idx  6
compile idx  7
compile idx  8
compile idx  9
compile idx  10
compile idx  11
compile idx  12
compile idx  13
compile idx  14

KeyboardInterrupt: 

In [ ]:
plt.plot(timeList, range(len(timeList)))
plt.xlabel("time(s)")
plt.ylabel("epoch size")

In [ ]:
result = {}
for i_iter in range(n_search):
    circList = []
    compiledCircList = []
    weightList = []
    selectedAIdList = np.random.randint(0, len(NAS_search_space), (n_layers,)).tolist()
    
    fciE = initCircuit(distance, selectedAIdList)
    
#     find optimal expert
    for i in range(n_experts):
        # get params
        params = []
        min_loss = 100000
        min_expert_id = 0
        params = modelParam.get_params(selectedAIdList, i)

        # calculate
        exp = 0
        for idx, compiledCirc in enumerate(compiledCircList):   
            exp += compiledCirc(modelParam.params).real*weightList[idx].real
        loss = exp
        if loss<min_loss:
            min_loss = loss
            min_expert_id = i
    expert_idx = min_expert_id
    # get params
    params = []
    params = modelParam.get_params(selectedAIdList, expert_idx) 
    
    # for each set
    exp = 0
    for idx, compiledCirc in enumerate(compiledCircList):   
        exp += compiledCirc(modelParam.params).real*weightList[idx].real
        pass

        
        
#     print(params)
    print(i_iter, selectedAIdList, fciE, exp.item())
    
    result['-'.join([str(x) for x in selectedAIdList])] = min_loss
    
time_end=time.time()

In [ ]:
print(time.time()-time_start)

In [ ]:
# result = {}
# print(n_search)
# for i_iter in range(n_search):
#     selectedAIdList = np.random.randint(0, len(NAS_search_space), (n_layers,)).tolist()
#     circList = []
#     compiledCircList = []
#     weightList = []
#     fciE = initCircuit(distance, selectedAIdList)
#     for i in range(n_experts):
# #         print(i)
#         # get params
#         params = []
#         min_loss = 100000
#         min_expert_id = 0
#         for j in range(n_layers):
#             r_idx = selectedAIdList[j] // len(CNOTs_space)
#             params.append(params_space[i, j, r_idx:r_idx+1])
#         params = torch.tensor(np.concatenate(params, axis=0), requires_grad=False)  

#         # calculate
#         exp = 0
#         for idx, compiledCirc in enumerate(compiledCircList):   
#             exp += compiledCirc(params).real*weightList[idx].real
# #     print(exp)
#         loss = exp
#         if loss<min_loss:
#             min_loss = loss
#             min_expert_id = i
#     print(i_iter, selectedAIdList, min_loss)
#     result['-'.join([str(x) for x in selectedAIdList])] = min_loss
    


In [ ]:
sorted_result = list(result.items())
sorted_result.sort(key=lambda x: x[1], reverse=True)
for idx, data in enumerate(sorted_result):
    print(idx, data[0],data[1].item())